In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Christian\AppData\Ro

True

In [4]:
import os
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


# # Change the current working directory to D: drive
os.chdir('D:/Projects/Github Repo/AI-Final-Project')

# Load the dataset
df = pd.read_csv(r"D:\Projects\Github Repo\AI-Final-Project\Nusrat_Dataset.csv", encoding='ISO-8859-1')
# Clean the data
def clean_text(text):
    if isinstance(text, str):  # Check if text is a string
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
        return text
    else:
        return ''

df['Tweets'] = df['Tweets'].apply(clean_text)
df.drop(df[df['Tweets'] == ''].index, inplace=True)
df.dropna(inplace=True)

In [7]:
df.groupby('Labels').describe()

Tweets         \
                  count unique   
Labels                           
atypical           1980   1965   
bipolar            2443   2434   
major depressive   2517   2515   
no                 1985   1984   
postpartum         3746   3739   
psychotic          2312   2303   

                                                                          
                                                                top freq  
Labels                                                                    
atypical                                        atypical depression    6  
bipolar                                            bipolar disorder    5  
major depressive                          major depressive disorder    3  
no                winner drumroll votes krystina jimenez congrat...    2  
postpartum                              think postpartum depression    3  
psychotic                                      psychotic depression    5

In [7]:
df.groupby('Labels').describe()

Tweets         \
                  count unique   
Labels                           
atypical           1980   1965   
bipolar            2443   2434   
major depressive   2517   2515   
no                 1985   1984   
postpartum         3746   3739   
psychotic          2312   2303   

                                                                          
                                                                top freq  
Labels                                                                    
atypical                                        atypical depression    6  
bipolar                                            bipolar disorder    5  
major depressive                          major depressive disorder    3  
no                winner drumroll votes krystina jimenez congrat...    2  
postpartum                              think postpartum depression    3  
psychotic                                      psychotic depression    5

In [8]:
df['Labels'].value_counts()

Labels
postpartum          3746
major depressive    2517
bipolar             2443
psychotic           2312
no                  1985
atypical            1980
Name: count, dtype: int64

In [30]:
df_postpartum = df[df['Labels'] == 'postpartum']
df_major = df[df['Labels'] == 'major depressive']
df_bipolar = df[df['Labels'] == 'bipolar']
df_psychotic = df[df['Labels'] == 'psychotic']
df_no = df[df['Labels'] == 'no']
df_atypical = df[df['Labels'] == 'atypical']

(1980, 2)

In [34]:
df_postpartum.shape

(3746, 2)

In [33]:
df_postpartum_downsampled = df_postpartum.sample(df_atypical.shape[0])
df_postpartum_downsampled.shape

(1980, 2)

In [43]:
df_major_downsampled = df_major.sample(df_atypical.shape[0])
df_major_downsampled.shape

(1980, 2)

In [42]:
df_bipolar_downsampled = df_bipolar.sample(df_atypical.shape[0])
df_bipolar_downsampled.shape

(1980, 2)

In [40]:
df_psychotic_downsampled = df_psychotic.sample(df_atypical.shape[0])
df_psychotic_downsampled.shape

(1980, 2)

In [41]:
df_no_downsampled = df_no.sample(df_atypical.shape[0])
df_no_downsampled.shape

(1980, 2)

In [44]:
df_balanced = pd.concat([df_postpartum_downsampled, df_major_downsampled, df_bipolar_downsampled, df_psychotic_downsampled, df_no_downsampled, df_atypical])
df_balanced.shape

(11880, 2)

In [46]:
df_balanced['Labels'].value_counts()

Labels
postpartum          1980
major depressive    1980
bipolar             1980
psychotic           1980
no                  1980
atypical            1980
Name: count, dtype: int64

In [2]:
!pip install ktrain
!pip install transformers

     ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
     --------------------------------------- 0.0/25.3 MB 217.9 kB/s eta 0:01:57
     --------------------------------------- 0.1/25.3 MB 435.7 kB/s eta 0:00:58
      --------------------------------------- 0.4/25.3 MB 3.1 MB/s eta 0:00:09
     - -------------------------------------- 0.9/25.3 MB 5.4 MB/s eta 0:00:05
     -- ------------------------------------- 1.5/25.3 MB 6.7 MB/s eta 0:00:04
     --- ------------------------------------ 2.0/25.3 MB 7.5 MB/s eta 0:00:04
     ---- ----------------------------------- 2.5/25.3 MB 8.1 MB/s eta 0:00:03
     ---- ----------------------------------- 3.1/25.3 MB 8.5 MB/s eta 0:00:03
     ----- ---------------------------------- 3.6/25.3 MB 8.9 MB/s eta 0:00:03
     ------ --------------------------------- 4.2/25.3 MB 9.2 MB/s eta 0:00:03
     ------- -------------------------------- 4.7/25.3 MB 9.4 MB/s eta 0:00:03
     ------- -------------------------------- 4.9/25.3 MB


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
